In [1]:
import cv2
import numpy as np
import skimage.io as io
from skimage.feature import canny
import matplotlib.pyplot as plt
from skimage.color import rgb2gray , rgb2hsv
from skimage.morphology import binary_erosion, binary_dilation, binary_closing,skeletonize, thin
from skimage.measure import find_contours
from skimage.draw import rectangle
from skimage.filters import threshold_otsu, threshold_mean, threshold_li, threshold_isodata, threshold_niblack , median
from skimage.morphology import binary_erosion, binary_dilation, binary_closing,skeletonize, thin , disk
from skimage.segmentation import flood, flood_fill
%matplotlib inline
%load_ext autoreload
%autoreload 2


In [2]:
def Gamma_Correction(img , c , gamma):
    img2 = np.copy(img)
    img2 = c*np.power(img , gamma)
    return img2

def erosion(img , n,m):
    img2 = np.copy(img)
    for i in range(int(n/2) , img.shape[1]-1-int(n/2) ):
        for j in range(int(m/2) , img.shape[0]-1-int(m/2) ):
            img2[j,i] = np.sum(img[j-int((m)/2) : j+int((m)/2) + n%2 , i-int((n)/2) : i+int((n)/2) + n%2 ])==n*n        
    return img2

def dilation(img , n,m):
    img2 = np.copy(img)
    for i in range(int(n/2) , img.shape[1]-1-int(n/2) ):
        for j in range(int(m/2) , img.shape[0]-1-int(m/2) ):
            img2[j,i] = np.sum(img[j-int((m)/2) : j+int((m)/2) + n%2 , i-int((n)/2) : i+int((n)/2) + n%2 ])>=1        
    return img2

def Median(img , n , m):
    img2 = np.copy(img)
    filt = np.ones((n,m))
    for i in range(int(n/2) , img.shape[1]-int(np.ceil(n/2))):
        for j in range(int(m/2) , img.shape[0]-int(np.ceil(m/2))):
            img2[j,i] = np.median(img[j-int(m/2) : j+int(m/2)+1 , i-int(n/2) : i+int(n/2)+1 ])
    return img2


In [3]:
def multi_color_space_model(rgba_frame , hsv_frame , ycbcr_frame):
    
    # Extracting the blue, red, green and alpha channels
    
    B = rgba_frame[:,:,0]
    G = rgba_frame[:,:,1]
    R = rgba_frame[:,:,2]
    A = rgba_frame[:,:,3]
    
    #Extracting the Hue, Saturation and vue channels
    
    H = hsv_frame[:,:,0]
    S = hsv_frame[:,:,1]
    V = hsv_frame[:,:,2]
    
    # Extracting the Y, Cr and Cb channels
    
    Y = ycbcr_frame[:,:,0]
    Cr = ycbcr_frame[:,:,1]
    Cb = ycbcr_frame[:,:,2]
    
    #     Applying  Thresholding using Log-Chromaticity color space 
    n_r = np.ma.divide(R, G)
    n_b = np.ma.divide(B, G)
    log_rg = np.ma.log( n_r )
    log_bg = np.ma.log( n_b )
    condition_1 = (log_rg>=0.15) & (log_rg<=1.1)
    condition_2 = (log_bg>=-4) & (log_bg<=0.3)
    mask_1 = condition_1 & condition_2
    
    # Extracting masks based on a combination of RGBA, HSV and YCrCb models for skin detection
    
    mask_rgb = (R>95)&(G>40)&(B>20)&(R>G)&(R>B)&(abs(R-G)>15)&(A>15)
    mask_Ycbcr = (Cr > 135)&(Cb>85)&(Y>80)&(Cr <= (1.5862*Cb)+20)&(Cr>=(0.3448*Cb)+76.2069)&(Cr >= (-4.5652*Cb)+234.5652)&(
                  Cr <= (-1.15*Cb)+301.75)&(Cr <= (-2.2857*Cb)+432.85)
    mask_hsv = ((0.0 <= H) & (H <= 50.0))&((0.23 <= S/255) & (S/255 <= 0.68))
#     print(mask_hsv)
    mask1 = mask_rgb & mask_Ycbcr
    mask2 = mask_rgb & mask_hsv
    
    return mask2 | mask1
#     return mask_hsv

In [4]:
def moving_area():
    
    frame_number = 0
    ret, rgb_frame = cap.read()
    rgb_frame = cv2.resize(rgb_frame, None, fx=1, fy=1, interpolation=cv2.INTER_AREA)
    gray_frame = rgb2gray(rgb_frame) 
    acc_frame = np.zeros((rgb_frame.shape[0] , rgb_frame.shape[1]))
    
    while (frame_number < 3):
        ret, cur_frame = cap.read()
        cur_frame = cv2.resize(cur_frame, None, fx=1, fy=1, interpolation=cv2.INTER_AREA)
        cur_frame = rgb2gray(cur_frame)
        acc_frame += cur_frame
        frame_number += 1 
        
    acc_frame /= frame_number
    diff_frame = abs(acc_frame-gray_frame)
    moving_region = np.zeros((rgb_frame.shape[0] , rgb_frame.shape[1]))
    moving_region[diff_frame > 0.2] = 1
    one_positions = np.argwhere(moving_region == 1)
    if(len(one_positions) > 0):
        Xmin = min(one_positions[:,0])
        Xmax = max(one_positions[:,0])
        Ymin = min(one_positions[:,1])
        Ymax = max(one_positions[:,1])
        moving_region[Xmin:Xmax , Ymin:Ymax] = 1
    
    return moving_region
    

In [5]:
cap = cv2.VideoCapture(0)

# Check if the webcam is opened correctly
if not cap.isOpened():
    raise IOError("Cannot open webcam")

ok_region = False
old_moving_region = -1 

fgbg = cv2.createBackgroundSubtractorMOG2()

while True:
    
    ret, rgb_frame = cap.read()
    rgb_frame = cv2.resize(rgb_frame, None, fx=1, fy=1, interpolation=cv2.INTER_AREA)
    rgb_frame = cv2.GaussianBlur(rgb_frame ,(7,7),cv2.BORDER_DEFAULT) 
    

    fgmask = fgbg.apply(rgb_frame)
    cv2.imshow('frame',fgmask)
    
    #if(not ok_region):
        #ok_region = True
        #old_moving_region = np.zeros((rgb_frame.shape[0] , rgb_frame.shape[1]))
#     face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')    
#     gray = cv2.cvtColor(rgb_frame, cv2.COLOR_BGR2GRAY)
#     faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    
#     for (x,y,w,h) in faces: 
#         # To draw a rectangle in a face  
#         cv2.rectangle(rgb_frame,(x,y),(x+w,y+h),(255,255,0),2)   
     
    
    # converting the rgb space to hsv space color
    hsv_frame = cv2.cvtColor(rgb_frame, cv2.COLOR_BGR2HSV)
    # converting the rgb space to YCbCr space color
    ycbcr_frame = cv2.cvtColor(rgb_frame, cv2.COLOR_BGR2YCR_CB)
    
    # converintg the rgb space to rgba spave 
    rgba_frame = cv2.cvtColor(rgb_frame, cv2.COLOR_BGR2BGRA)
    
    moving_region = moving_area()
    #if(np.sum(moving_region) <= 1000):
    #    moving_region = old_moving_region
    #old_moving_region = moving_region
    
    # the final mask to extract the binary image
    mask = multi_color_space_model(rgba_frame , hsv_frame , ycbcr_frame)
    
    binary_frame2 = np.zeros((rgb_frame.shape[0] , rgb_frame.shape[1]))
    binary_frame2[mask] = 1
    
#     for (x,y,w,h) in faces: 
#         # To draw a rectangle in a face  
#         binary_frame2[y-30:y+h+30 , x:x+w] = 0
        
    #removing noise
    eroded_frame = binary_erosion(binary_frame2 , np.ones((3,3)))
    dilated_frame = binary_dilation(eroded_frame , np.ones((10,5)))
    
    binary_frame2 = np.zeros((rgb_frame.shape[0] , rgb_frame.shape[1]))
    binary_frame2[dilated_frame] = 1
         
    binary_frame2 = median(binary_frame2 , disk(1) ) 
    
    contours, hierarchy = cv2.findContours(binary_frame2, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    bounding_boxes = list()
    mask_face = np.zeros((rgb_frame.shape[0] , rgb_frame.shape[1]))
    for contour in contours:
        x,y,w,h = cv2.boundingRect(contour)
        print(x,y,w,h)
        if( ( (float(w)/h <= 0.9 and float(h)/w >= 1.3) or (float(h)/w <= 0.5 and float(w)/h >= 1.3 ) )):
            mask_face[y:y+h , x:x+w] = 1
            cv2.drawContours(rgb_frame, contour, -1, (0, 255, 0), 3)
    
    #cv2.imshow('binary canny detection', final_frame)
#     cv2.imshow('canny detection', cannyImg )

    frame_with_hand_only = np.zeros((rgb_frame.shape[0] , rgb_frame.shape[1]))
    frame_with_hand_only[(binary_frame2>=1) & (mask_face>=1) &(fgmask>=1) ] = 1
    
    cv2.imshow('Input', rgb_frame)
    cv2.imshow('Output1', binary_frame2)
    cv2.imshow('mask_face', mask_face)
    cv2.imshow('Final Output', frame_with_hand_only)
    #cv2.imshow('moving region',moving_region)
    
    
    c = cv2.waitKey(1)
    if c == 27:
        break

cap.release()
cv2.destroyAllWindows()


/home/ahmed/anaconda3/lib/python3.7/site-packages/skimage/util/dtype.py:135: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))


391 464 5 11
14 446 6 11
18 407 5 10
0 404 4 13
3 349 11 13
0 330 10 18
38 319 6 11
313 310 18 32
38 305 7 12
334 298 6 10
11 294 23 32
291 280 11 16
270 264 11 16
50 261 8 12
338 254 17 33
306 239 8 23
245 239 17 20
319 221 6 13
123 220 10 15
141 208 6 11
196 206 8 14
184 204 5 10
236 199 6 12
148 195 6 14
159 173 86 65
200 183 4 7
300 170 5 10
265 166 26 24
244 156 20 35
240 156 6 11
291 154 6 11
248 141 10 11
327 132 9 18
257 116 18 19
310 106 6 12
270 98 48 51
195 78 22 57
142 474 6 6
345 473 5 7
108 470 10 10
169 468 15 12
349 459 5 10
150 456 6 10
65 456 5 10
333 454 6 12
381 452 11 28
172 448 8 12
182 444 14 26
368 439 13 35
336 439 6 11
250 438 18 17
290 423 15 17
55 422 24 26
41 417 5 10
68 406 8 13
369 400 11 14
220 392 5 10
373 387 5 10
36 385 7 11
59 377 6 11
365 368 8 14
308 364 8 14
40 360 18 33
149 354 12 17
303 347 10 11
128 325 5 10
323 323 10 11
58 323 16 17
119 316 6 11
143 311 17 21
344 302 5 10
0 302 56 178
18 468 4 4
17 423 7 6
7 404 6 11
294 300 6 11
302 288 30 5

464 469 6 11
597 410 11 15
619 392 8 13
0 382 14 37
563 372 10 13
620 364 6 12
25 346 7 14
574 336 40 65
145 229 9 11
156 223 5 10
172 190 15 28
215 183 6 10
199 183 9 10
240 173 40 61
311 111 6 12
230 108 13 17
254 105 58 47
224 80 9 27
329 29 7 21
313 22 12 15
340 21 10 14
275 18 12 19
266 7 5 10
288 0 16 33
189 0 41 110
206 46 6 7
464 462 6 18
556 435 6 11
585 418 18 23
549 415 15 16
622 410 5 10
567 410 9 12
0 410 5 13
571 392 8 11
618 384 7 11
9 382 7 12
556 380 14 19
592 379 21 21
582 378 8 12
602 355 5 10
575 341 24 29
280 225 5 10
157 222 5 10
172 191 16 28
243 172 32 57
232 110 12 21
319 108 13 14
342 102 7 12
255 100 57 51
224 97 6 10
331 91 8 12
226 81 7 14
328 72 10 13
296 19 5 10
187 16 44 99
207 32 8 18
314 15 17 32
211 0 17 16
194 0 16 15
465 474 5 6
579 423 8 11
589 419 9 12
616 396 7 11
561 395 12 14
556 383 7 15
11 379 6 13
580 374 16 37
595 352 45 43
570 350 16 20
572 331 18 18
46 300 5 10
151 219 14 20
168 200 16 24
216 183 5 10
241 171 35 49
326 106 10 14
339 102 9

494 457 7 23
125 390 7 12
3 373 5 11
540 331 100 118
609 396 9 14
615 374 25 21
575 354 5 7
141 327 9 20
102 323 25 94
134 318 52 74
66 318 10 11
25 307 5 10
186 305 7 11
344 304 7 13
105 287 5 10
182 283 6 11
198 279 14 21
94 264 9 12
23 264 9 22
405 246 10 14
365 242 8 20
280 237 11 23
23 237 5 11
138 233 56 91
199 225 9 13
51 223 5 11
254 217 19 25
274 216 8 11
218 216 7 12
218 209 140 103
321 260 11 10
251 207 7 11
53 195 15 27
180 189 14 27
341 161 41 37
384 160 42 35
95 138 45 152
115 136 13 30
48 134 7 14
92 126 15 68
75 121 20 83
87 163 4 5
184 0 103 119
494 457 8 23
3 451 5 10
636 390 4 12
5 380 5 10
542 332 98 116
570 424 6 6
574 408 5 4
590 400 4 3
604 394 5 4
574 374 12 12
126 318 58 82
342 302 8 13
103 297 50 119
26 292 6 18
156 281 13 19
24 277 5 11
195 276 14 37
81 265 7 10
215 248 122 64
245 293 4 4
24 248 16 15
50 247 11 12
155 235 39 46
229 232 9 14
23 229 7 12
261 228 17 19
362 226 7 10
382 224 5 10
249 224 8 14
183 224 6 10
333 218 41 48
296 217 7 13
310 209 25 51
2

494 456 6 24
542 330 98 118
566 402 13 26
579 372 4 3
126 290 11 15
84 267 18 82
304 264 6 10
199 259 6 10
252 244 15 20
143 244 23 27
119 239 7 12
207 231 16 31
140 230 8 13
93 230 5 10
349 226 12 12
260 224 10 16
336 223 38 48
384 217 8 11
214 216 8 13
138 216 6 11
195 212 144 101
105 195 29 33
114 213 3 3
116 207 3 5
135 193 10 21
224 191 6 17
168 190 35 135
228 168 7 12
340 162 87 44
397 184 5 4
197 140 17 43
135 107 60 106
186 0 100 117
492 456 7 24
545 332 95 115
571 428 6 5
576 405 6 8
567 393 5 8
578 384 3 3
619 381 8 12
627 376 4 4
125 306 7 18
85 289 17 60
134 275 17 20
97 272 7 15
216 258 7 11
225 253 8 14
345 249 12 23
273 248 14 14
358 241 15 23
139 228 14 26
338 225 21 29
250 215 7 11
218 205 7 11
136 204 7 11
233 199 6 10
92 194 36 72
97 237 4 8
341 163 33 38
367 161 59 39
397 182 3 3
129 132 210 193
213 296 5 4
188 283 3 4
261 282 12 11
320 264 11 6
164 255 5 4
187 126 10 36
186 0 98 116
492 463 7 17
593 428 9 12
541 331 99 117
571 407 10 29
629 378 3 10
630 371 6 7
582

...

21 449 11 23
34 430 9 18
44 421 5 11
551 397 7 13
58 371 12 32
80 346 5 10
86 337 14 15
552 333 57 67
565 373 7 5
576 353 4 5
309 299 6 10
222 298 7 11
529 292 12 18
500 268 11 22
462 268 36 48
490 261 5 10
323 242 5 11
328 228 6 12
564 213 19 16
101 212 219 131
480 193 14 22
502 189 7 11
555 187 11 32
593 173 26 26
610 172 22 43
218 160 98 52
554 120 12 23
460 88 7 23
555 86 6 13
541 59 16 23
468 45 94 144
531 157 13 18
514 133 4 5
184 0 100 117
209 16 4 4
17 433 25 47
57 394 5 10
256 370 5 11
61 366 9 23
80 346 6 11
549 337 61 71
567 354 23 21
90 334 14 16
307 298 8 11
318 296 5 10
485 293 5 10
248 287 10 14
222 287 5 10
442 283 32 45
474 272 13 20
474 254 10 14
463 251 8 13
545 237 8 13
473 235 7 13
108 212 208 127
217 254 3 3
255 243 3 3
274 240 4 6
533 210 20 29
564 207 17 22
472 193 20 22
538 192 5 10
576 169 34 38
596 179 5 5
217 157 100 57
446 150 30 63
452 190 8 10
441 138 16 16
443 97 14 22
523 88 8 11
532 85 7 13
486 71 8 20
521 63 7 13
467 62 76 133
495 158 4 3
499 156 3 4


36 474 6 6
27 428 18 31
559 399 9 13
548 398 6 12
556 376 9 13
53 363 19 51
82 336 22 19
560 334 50 68
446 326 16 14
434 299 13 20
305 297 10 11
426 295 6 11
247 295 7 11
318 294 13 12
332 292 6 10
509 275 9 13
389 270 32 84
443 258 6 11
313 250 5 10
105 221 211 121
267 239 8 8
189 214 25 51
324 213 7 24
494 192 32 66
212 156 97 57
367 139 27 73
463 123 6 11
382 123 119 160
469 219 6 6
419 218 4 10
469 188 3 6
472 118 8 18
408 108 17 33
436 100 16 18
185 0 99 114
238 17 7 7
16 474 5 6
29 433 22 23
45 410 11 21
563 394 15 14
549 385 9 18
57 373 11 30
558 334 51 56
570 367 6 8
395 333 6 11
448 330 6 11
458 321 9 13
451 308 5 10
222 299 7 13
303 297 12 13
321 295 5 10
392 291 29 62
412 276 27 56
514 259 12 20
468 243 23 32
475 222 5 11
383 219 7 12
80 213 239 144
279 240 21 10
325 208 7 24
502 194 30 64
374 182 9 19
211 158 101 56
364 142 18 38
385 118 100 135
411 196 5 4
436 101 18 37
185 0 98 117
223 0 5 9
16 466 10 14
32 422 17 30
50 372 18 49
279 341 5 10
273 334 5 10
548 333 62 75
57

24 463 7 16
535 436 6 11
54 408 7 13
355 353 11 13
80 338 16 25
543 334 66 94
556 388 9 12
326 259 7 11
398 243 6 10
328 231 84 80
365 284 3 3
409 224 20 36
98 207 250 138
186 0 98 115
246 5 3 3
29 440 15 26
45 420 10 18
59 403 7 12
568 397 8 14
543 388 20 40
65 382 11 17
558 332 57 63
586 355 6 12
393 272 19 19
416 231 11 28
79 207 313 157
366 267 4 9
335 262 6 8
184 0 98 116
24 463 6 15
32 447 7 12
572 384 9 13
535 384 38 63
64 380 8 18
555 337 60 46
585 357 4 5
343 336 8 13
391 258 14 21
229 250 6 11
391 199 29 57
80 198 310 164
311 246 4 5
312 0 5 6
185 0 99 115
137 473 8 7
22 462 7 15
536 422 18 24
364 418 8 13
47 418 6 12
544 390 21 27
62 374 10 24
296 343 6 14
81 336 20 22
555 335 59 68
575 365 6 11
572 357 6 6
103 327 13 15
121 287 50 45
323 214 5 10
174 192 220 150
348 262 3 3
380 188 30 56
186 0 97 116
228 20 5 6
21 447 16 33
539 433 5 10
39 421 12 32
61 374 13 29
545 334 69 90
557 396 9 10
91 332 18 18
355 221 6 10
112 190 268 153
332 246 5 3
369 177 29 50
186 0 92 112
213 6

14 473 10 7
535 435 7 13
5 413 26 41
581 404 7 10
37 397 6 11
66 359 6 11
544 332 72 106
120 312 7 13
207 287 6 13
263 283 40 66
84 203 200 155
288 192 22 53
295 218 4 5
168 150 99 64
184 0 95 117
9 464 7 11
3 434 13 19
21 416 7 13
536 333 80 112
561 407 3 3
571 378 5 5
316 297 5 10
301 296 10 12
268 255 31 92
80 203 202 156
285 192 24 54
293 217 3 3
167 150 95 64
185 0 95 116
245 0 3 6
133 472 10 8
0 423 24 36
29 404 9 15
66 359 6 11
535 332 81 115
554 396 5 7
589 365 3 3
318 295 5 10
263 270 40 74
87 203 196 153
284 192 24 54
165 151 93 63
183 0 97 119
0 437 13 24
17 411 16 27
34 399 8 14
535 332 83 115
283 320 5 11
297 268 7 12
80 190 230 169
220 232 14 11
292 215 5 7
167 148 98 63
182 0 97 118
7 438 6 11
19 423 5 10
552 422 5 10
535 421 10 27
25 415 5 10
592 384 17 19
56 374 5 10
63 360 9 13
544 336 73 87
549 405 4 3
568 398 4 4
556 394 5 7
318 295 5 10
337 291 5 10
266 286 38 64
206 281 9 15
284 258 10 27
85 201 202 156
184 269 8 14
182 265 4 5
233 232 5 4
289 193 23 55
296 217 4 

7 431 8 15
65 359 6 10
535 334 74 113
567 368 8 10
572 354 8 7
314 296 7 10
84 279 110 79
223 268 16 13
287 257 7 12
256 256 45 95
270 246 5 10
170 197 101 77
275 191 29 54
163 144 93 60
311 0 6 6
184 0 99 116
0 437 11 21
25 411 5 10
62 361 8 13
536 337 68 109
571 385 5 9
574 377 3 4
578 359 9 13
303 299 6 10
206 290 6 11
80 278 118 82
255 252 48 94
171 191 132 81
286 215 7 6
163 145 91 59
311 0 5 5
184 0 99 117
204 17 4 4
0 442 6 12
9 430 7 16
34 398 5 11
594 393 5 10
586 385 7 11
58 358 11 21
536 331 73 117
304 299 6 10
193 293 5 10
80 279 112 81
249 256 51 91
170 202 105 74
208 224 13 11
274 191 29 52
285 214 6 7
162 145 92 60
234 161 5 6
184 0 100 118
43 474 6 6
0 443 6 16
536 428 8 17
7 424 18 25
567 411 9 12
24 398 16 25
606 387 5 10
49 377 6 11
542 335 62 97
549 402 3 3
565 392 10 6
200 305 5 11
81 280 113 79
255 276 43 70
226 270 11 12
289 266 5 11
278 265 5 10
170 197 105 82
264 238 5 5
274 189 31 57
162 145 90 60
185 0 98 118
0 439 8 18
535 433 8 14
42 431 6 13
26 410 5 10
30

In [6]:
# cap = cv2.VideoCapture(0)
# if not cap.isOpened():
#     raise IOError("Cannot open webcam")
# while True:
#     ret, rgb_frame = cap.read()
#     rgb_frame = cv2.resize(rgb_frame, None, fx=1, fy=1, interpolation=cv2.INTER_AREA)
#     # converting the rgb space to hsv space color
#     hsv_frame = cv2.cvtColor(rgb_frame, cv2.COLOR_BGR2HSV)
#     # converting the rgb space to YCbCr space color
#     ycbcr_frame = cv2.cvtColor(rgb_frame, cv2.COLOR_BGR2YCR_CB)
    
#     # converintg the rgb space to rgba spave 
#     rgba_frame = cv2.cvtColor(rgb_frame, cv2.COLOR_BGR2BGRA)
#      # the final mask to extract the binary image
#     mask = multi_color_space_model(rgba_frame , hsv_frame , ycbcr_frame)
#     binary_frame = np.zeros((rgb_frame.shape[0] , rgb_frame.shape[1]) , dtype=np.uint8)
#     binary_frame2 = np.zeros((rgb_frame.shape[0] , rgb_frame.shape[1]))
#     binary_frame2[mask] = 1
#     binary_frame[mask] = 1
# #   binary_frame = median(binary_frame , disk(2) )
#     h, w = binary_frame.shape[:2]
#     maskk = np.zeros((h+2, w+2), np.uint8)
#     im_floodfill = np.copy(binary_frame)
    
#     # Floodfill from point (0, 0)
#     cv2.floodFill(im_floodfill, maskk, (0,0), 255);

#     # Invert floodfilled image
#     im_floodfill_inv = cv2.bitwise_not(im_floodfill)
 
#     # Combine the two images to get the foreground.
#     im_out = binary_frame | im_floodfill_inv
#     im_out = median(im_out , disk(2) )
#     dilated_img = binary_dilation(im_out , np.ones((8,4))) 
#     im_out =  np.zeros((rgb_frame.shape[0] , rgb_frame.shape[1]))
#     im_out[dilated_img] = 1
#     cv2.imshow('Input', rgb_frame)
#     cv2.imshow('Output2', im_out)
#     cv2.imshow('Output1', binary_frame2)
#     c = cv2.waitKey(1)
#     if c == 27:
#         break

# cap.release()
# cv2.destroyAllWindows()
    

In [7]:
# import numpy as np
# import cv2

# cap = cv2.VideoCapture(0)

# fgbg = cv2.createBackgroundSubtractorMOG2()

# while(1):
#     ret, frame = cap.read()
#     if ret == True:
#         fgmask = fgbg.apply(frame)

#         cv2.imshow('frame',fgmask)
#         k = cv2.waitKey(30) & 0xff
#         if k == 27:
#             break
#     else:
#         break

# cap.release()
# cv2.destroyAllWindows()
